In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

In [ ]:
def handle_file_upload(file):
    initial_msg = gen_initial_msg(file.name)
    try:
        # Initial assistant response
        model_resp = generate_responses_from_inf(hf_token, initial_msg, "Qwen/Qwen2.5-Coder-32B-Instruct", 5000)
        conv_history = [("assistant", model_resp)] 
        msg_history = [{"role": "assistant", "content": model_resp}]  
        return conv_history, msg_history
    except Exception as e:
        if e.response.status_code in [422, 500]:
            return "Model Busy!!! Please try after sometime.", None
        else:
            return "An unexpected error occurred.", None

def handle_user_message(user_msg, history):
    # Append the assistant's previous response and the new user message to the conversation history
    assistant_msg = generate_responses_from_inf(hf_token, history + [("user", user_msg)], "Qwen/Qwen2.5-Coder-32B-Instruct", 5000)
    
    # Append the new message to the history
    history.append({"role": "assistant", "content": assistant_msg})
    history.append({"role": "user", "content": user_msg})
    
    return ("assistant", assistant_msg), history

# Gradio Interface
with gr.Blocks() as app:
    # Adding a header for the app
    gr.Markdown("""
    <div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;">
        <h1 style="text-align: center;">Chat with Assistant</h1>
    </div>
    """, elem_id="header")

    with gr.Row():
        with gr.Column(scale=1):
            file_upload = gr.File(label="Upload Excel File", file_types=[".xlsx"])
            retry_btn = gr.Button("Retry", visible=False)
            
        with gr.Column(scale=2):
            # Set up the chatbot
            chatbot = gr.Chatbot()

    # Connect file upload action to the handle function
    file_upload.change(handle_file_upload, inputs=file_upload, outputs=[chatbot, retry_btn])
    
    # User input and message submission
    user_input = gr.Textbox(label="Your Message", placeholder="Type here...")
    user_input.submit(handle_user_message, inputs=[user_input, chatbot], outputs=[chatbot, user_input])

    # Retry button to reset chat interface
    retry_btn.click(handle_file_upload, inputs=file_upload, outputs=[chatbot, retry_btn])

# CSS styling to style the button and chat interface
app.css = """
#retry-btn {
    background-color: #4CAF50;
    color: white;
    width: 200px;
    height: 50px;
    border-radius: 10px;
    font-size: 16px;
}

#chat-interface {
    background-color: #E8E8E8;
    border-radius: 10px;
    width: 100%;
    height: 500px;
    padding: 10px;
    overflow-y: scroll;
}
"""

# Launch the app
app.launch()

In [ ]:
import gradio as gr

# Initialize an empty list to maintain chat history
chat_history = []

# Function to handle chat history
def chatbot_history(user_input):
    global chat_history  # Use the global chat_history list
    
    # Append user input to the chat history
    chat_history.append(("User", user_input))
    
    # Generate a response (you can replace this with more complex logic)
    bot_response = f"Bot: {user_input[::-1]}"  # Reverse input for fun
    
    # Append bot response to chat history
    chat_history.append(("Bot", bot_response))
    
    # Create a formatted string of the conversation history
    conversation_history = ""
    
    # Add user and bot messages with flexbox styling for alignment
    for speaker, message in chat_history:
        if speaker == "User":
            conversation_history += f'''
                <div style="display: flex; justify-content: flex-end; margin-bottom: 5px;">
                    <div style="background-color: lightblue; color: blue; padding: 8px; border-radius: 10px; max-width: 60%; word-wrap: break-word;">
                        <b>{speaker}:</b> {message}
                    </div>
                </div>
            '''
        else:
            conversation_history += f'''
                <div style="display: flex; justify-content: flex-start; margin-bottom: 5px;">
                    <div style="background-color: lightgreen; color: green; padding: 8px; border-radius: 10px; max-width: 60%; word-wrap: break-word;">
                        <b>{speaker}:</b> {message}
                    </div>
                </div>
            '''
    
    # Return the formatted conversation history and clear the input field
    return conversation_history, ""

# Create Gradio interface using Blocks
with gr.Blocks() as demo:
    input_text = gr.Textbox(label="Enter your message", lines=1)
    submit_button = gr.Button("Submit")
    output_text = gr.HTML(label="Chatbot History")

    # Set up the submit button to trigger the chatbot_history function
    submit_button.click(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])

# Launch the app
demo.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
